In [13]:
cd("C:\\Users\\zlosc\\Documents\\GitHub\\StateSpaceDynamics.jl")

using Pkg
Pkg.activate("./benchmarking")

using StateSpaceDynamics
using Profile
using LinearAlgebra
using Plots
using Random

const SSD = StateSpaceDynamics

  Activating project at `C:\Users\zlosc\Documents\GitHub\StateSpaceDynamics.jl\benchmarking`


StateSpaceDynamics

In [34]:
A

2×2 Matrix{Float64}:
 0.99  0.01
 0.05  0.95

In [33]:
πₖ

2-element Vector{Float64}:
 0.5
 0.5

In [41]:

# Define the output dimensionality of the HMM
output_dim = 1

# # Define the transition matrix and the initial state distribution
# A = [0.99 0.01; 0.05 0.95];
# πₖ = [0.5; 0.5]

A = rand(6,6)
A .= A ./ sum(A, dims=2)

πₖ = rand(6)
πₖ .= πₖ ./ sum(πₖ)


# Initialize the emission models
μ_1 = rand(output_dim)
Σ_1 = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(output_dim=output_dim, μ=μ_1, Σ=Σ_1)

μ_2 = rand(output_dim)
Σ_2 = 0.2 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(output_dim=output_dim, μ=μ_2, Σ=Σ_2)

emission_3 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=0.1 * Matrix{Float64}(I, output_dim, output_dim))
emission_4 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=1.0 * Matrix{Float64}(I, output_dim, output_dim))
emission_5 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=3.0 * Matrix{Float64}(I, output_dim, output_dim))
emission_6 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=2.7 * Matrix{Float64}(I, output_dim, output_dim))



# The general HMM constructor is used as follows
model = HiddenMarkovModel(K=6, B=[emission_1, emission_2, emission_3, emission_4, emission_5, emission_6], A=A, πₖ=πₖ)

HiddenMarkovModel{Float64, Vector{Float64}, Matrix{Float64}, Vector{GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}}}([0.159870383823143 0.14360056136570054 … 0.29110824711432065 0.11569180675840415; 0.2345600336078144 0.06509953748637912 … 0.231496784439123 0.07660658178656968; … ; 0.18675945556754384 0.05870772970770826 … 0.27958337842246234 0.262396357806585; 0.18449948870143254 0.22760702880870895 … 0.2125642876421356 0.015322108204210094], GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}[GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}(1, [0.8125904375146918], [0.1;;]), GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}(1, [0.16338808967634988], [0.2;;]), GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}(1, [0.02385844350513089], [0.1;;]), GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}(1, [0.9556846947972149], [1.0;;]), GaussianEmission{Float64, Vector{Float64}, Matrix{Float64}}(1, [0.7010984613851785], [3.0;;]), Ga

In [ ]:

# Define number of samples to generate 
n=1000

# Use the SSD sampling function and store the hidden states and emission data
true_labels, data = rand(model, n=n)


([5, 6, 1, 4, 2, 4, 2, 2, 1, 3  …  5, 6, 4, 6, 2, 4, 4, 5, 5, 4], [-0.7138424124395033 1.3537535292164744 … 0.3803284767245012 0.1517450095088424])

In [48]:
emission_1 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=0.001 * Matrix{Float64}(I, output_dim, output_dim))
emission_2 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=5.23 * Matrix{Float64}(I, output_dim, output_dim))
emission_3 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=9.1 * Matrix{Float64}(I, output_dim, output_dim))
emission_4 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=0.88 * Matrix{Float64}(I, output_dim, output_dim))
emission_5 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=2.1 * Matrix{Float64}(I, output_dim, output_dim))
emission_6 = GaussianEmission(output_dim=output_dim, μ=rand(output_dim), Σ=3.1 * Matrix{Float64}(I, output_dim, output_dim))

A = rand(6,6)
A .= A ./ sum(A, dims=2)

πₖ = rand(6)
πₖ .= πₖ ./ sum(πₖ)

test_model = HiddenMarkovModel(K=6, B=[emission_1, emission_2, emission_3, emission_4, emission_5, emission_6], A=A, πₖ=πₖ)

# Fit the HMM to the data using the SSD fit!() function
@profview lls = fit!(test_model, data, max_iters=100, tol=1e-100)

# # Show that we can properly recover the true parameters
# println("Recovered Emission 1?: ", isapprox(test_model.B[1].μ, model.B[1].μ, atol=0.1) || isapprox(test_model.B[1].μ, model.B[2].μ, atol=0.1))
# println("Recovered Emission 2?: ",isapprox(test_model.B[2].μ, model.B[2].μ, atol=0.1) || isapprox(test_model.B[2].μ, model.B[1].μ, atol=0.1))

# plot(lls)
# title!("Log-likelihood over EM Iterations")
# xlabel!("EM Iteration")
# ylabel!("Log-Likelihood")